In [1]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Load data from Excel sheet
excel_file_path = r'C:\Users\anush\Downloads\category.xlsx'
df = pd.read_excel(excel_file_path)

# Create a mapping between image file names and categories
image_file_to_category = dict(zip(df['Photo'], df['Category']))

# Load images and their corresponding labels
image_data = []
labels = []
images_folder = r'C:\Users\anush\Downloads\images\Photos_Walking Audit'
for image_file in os.listdir(images_folder):
    if image_file in image_file_to_category:
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))  # Resize image to (224, 224) or any other desired size
        image = image.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
        image_data.append(image)
        labels.append(image_file_to_category[image_file])

# Convert lists to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [8]:
train_labels


array(['Waste', 'Footpath Quality', 'Waste', 'Waste', 'Waste',
       'Encroachment', 'Encroachment', 'Encroachment', 'Obstruction',
       'Footpath Quality', 'Encroachment', 'Encroachment',
       'Footpath Quality', 'Obstruction', 'Waste', 'Footpath Quality',
       'Footpath Quality', 'Encroachment', 'Encroachment',
       'Footpath Quality', 'Encroachment', 'Encroachment',
       'Footpath Quality', 'Footpath Quality', 'Encroachment',
       'Footpath Quality', 'Waste', 'Waste', 'Encroachment',
       'Footpath Quality', 'Encroachment', 'Obstruction',
       'Footpath Quality', 'Waste', 'Footpath Quality', 'Waste',
       'Footpath Quality', 'Footpath Quality', 'Footpath Quality',
       'Footpath Quality', 'Footpath Quality', 'Unsafe Zone',
       'Footpath Quality', 'Waste', 'Encroachment', 'Footpath Quality',
       'Footpath Quality', 'Footpath Quality', 'Encroachment', 'Waste',
       'Footpath Quality', 'Footpath Quality', 'Obstruction',
       'Encroachment', 'Footpath Qual

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


model = Sequential()


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())

# Fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5)) 

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


num_categories = 10  
model.add(Dense(num_categories, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

Epoch 1/10


ValueError: in user code:

    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\anush\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C